# Deep Past Initiative – Masked thign

Idea:
use byt5 to fill in blanks in akkadian text, then move to the other training file to translate akkadian to english.


### TODO:Set up the huggingface mask-fill thing
1. Run it with just the intact examples and see how it does.
<!-- 2. Run it with splits  -->
Perhaps remove everything in this notebook and just start from scratch with the huggingface guide's stuffs

https://huggingface.co/tasks/fill-mask
https://huggingface.co/docs/transformers/tasks/masked_language_modeling


#### The byt5 stuff requires special setup it seems:
https://discuss.huggingface.co/t/how-can-i-pretrain-t5-model/168563/4?utm_source=chatgpt.com
https://aiml.com/t5-architecture-explained-encoder-decoder-model-comparison/
How to do mask prediction with ByT5? https://github.com/huggingface/transformers/issues/16303

#### Custom data collator
https://towardsdatascience.com/data-collators-in-huggingface-a0c76db798d2/

#### How would I change it from seq2seq to masked language modelling?
- https://chatgpt.com/share/69823f1b-e260-8007-89f0-7579204c61f6
    - https://mbrenndoerfer.com/writing/span-corruption-t5-pretraining-objective?utm_source=chatgpt.com

In [1]:
!pip install evaluate sacrebleu 
# !pip install pytorch --no-cache-dir --ignore-installed

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.8/100.8 kB 8.1 MB/s eta 0:00:00


In [2]:
from dataclasses import dataclass
from transformers import BatchEncoding
import torch
import numpy as np
class DataCollatorForT5MLM:
    tokenizer: any
    noise_density: float = 0.15   # T5 default
    mean_noise_span_length: float = 3.0  # T5 default
    input_length: int = 512       # final encoder length after corruption
    target_length: int = 114      # final decoder label length (computed by helper below)
    pad_token_id: int = 0
    decoder_start_token_id: int = 0

    def __post_init__(self):
        # If you don't know target_length, compute both from the helper given your desired inputs_length
        in_expanded, tgt_len = self.compute_input_and_target_lengths(
            self.input_length, self.noise_density, self.mean_noise_span_length
        )
        # We want batches fed in already at the "expanded" length before compression.
        self.expanded_input_length = in_expanded
        # Allow caller override of target_length if set; otherwise use the helper.
        if self.target_length is None:
            self.target_length = tgt_len

    # === Canonical helper from HF's T5 span-corruption example (ported to PyTorch use) ===
    # Ref: run_t5_mlm_flax.py -> compute_input_and_target_lengths
    def compute_input_and_target_lengths(self, inputs_length, noise_density, mean_noise_span_length):
        # Extra tokens per span: one sentinel on inputs, one sentinel on targets.
        def _tokens_length_to_spans(tokens_length):
            num_noise_tokens = int(np.round(tokens_length * noise_density))
            num_noise_tokens = min(max(num_noise_tokens, 1), tokens_length - 1)
            num_noise_spans = int(np.round(num_noise_tokens / mean_noise_span_length))
            num_noise_spans = max(num_noise_spans, 1)
            return num_noise_tokens, num_noise_spans

        num_noise_tokens, num_noise_spans = _tokens_length_to_spans(inputs_length - 1)  # minus EOS
        # Inputs: non-noise tokens + one sentinel per noise span + EOS
        input_len = (inputs_length - num_noise_tokens) + num_noise_spans + 1
        # Targets: noise tokens + one sentinel per noise span + EOS
        target_len = num_noise_tokens + num_noise_spans + 1
        # We need to invert this to get the *expanded* raw length that, after corruption, compresses to `inputs_length`.
        # Use inputs_length as the desired "post-compression" length and solve for expanded length iteratively.
        # Small search around desired length works well in practice.
        desired = inputs_length
        for raw in range(desired, desired + 1024):
            n_noise, n_spans = _tokens_length_to_spans(raw - 1)
            comp = (raw - n_noise) + n_spans + 1
            if comp == desired:
                tgt = n_noise + n_spans + 1
                return raw, tgt
        # Fallback if not found
        return inputs_length * 2, target_len

    # === Core collator ===
    def __call__(self, examples):
        # Expect each example to contain "input_ids" already tokenized to expanded_input_length
        # Keep masking logic in NumPy for speed, convert once to torch.
        np_inputs = np.array([ex["input_ids"] for ex in examples], dtype=np.int32)
        batch_size, expanded_len = np_inputs.shape
        if expanded_len != self.expanded_input_length:
            raise ValueError(
                f"Got expanded length {expanded_len}, expected {self.expanded_input_length}. "
                f"Pad/trim your dataset to the collator's expanded length first."
            )

        mask_indices = np.asarray(
            [self.random_spans_noise_mask(expanded_len) for _ in range(batch_size)],
            dtype=bool,
        )
        labels_mask = ~mask_indices

        input_ids_sentinel = self.create_sentinel_ids(mask_indices.astype(np.int8))
        labels_sentinel = self.create_sentinel_ids(labels_mask.astype(np.int8))

        enc_inputs = self.filter_input_ids(np_inputs, input_ids_sentinel)   # shape: (B, input_length)
        dec_labels = self.filter_input_ids(np_inputs, labels_sentinel)      # shape: (B, target_length)

        if enc_inputs.shape[-1] != self.input_length:
            raise ValueError(f"`input_ids` length {enc_inputs.shape[-1]} != configured {self.input_length}")
        if dec_labels.shape[-1] != self.target_length:
            raise ValueError(f"`labels` length {dec_labels.shape[-1]} != configured {self.target_length}")

        # Convert once to torch. Let the model derive decoder_input_ids from labels internally.
        batch = {
            "input_ids": torch.tensor(enc_inputs, dtype=torch.long),
            "attention_mask": (torch.tensor(enc_inputs, dtype=torch.long) != self.pad_token_id).long(),
            "labels": torch.tensor(dec_labels, dtype=torch.long),
        }
        return BatchEncoding(batch)

    # === Utilities copied from HF example (with >0 filter) ===
    # Ref: run_t5_mlm_flax.py -> create_sentinel_ids / filter_input_ids / random_spans_noise_mask
    def create_sentinel_ids(self, mask_indices):
        start_indices = mask_indices - np.roll(mask_indices, 1, axis=-1) * mask_indices
        start_indices[:, 0] = mask_indices[:, 0]
        sentinel_ids = np.where(start_indices != 0, np.cumsum(start_indices, axis=-1), start_indices)
        sentinel_ids = np.where(sentinel_ids != 0, (len(self.tokenizer) - sentinel_ids), 0)
        sentinel_ids -= mask_indices - start_indices
        return sentinel_ids

    def filter_input_ids(self, input_ids, sentinel_ids):
        batch_size = input_ids.shape[0]
        input_ids_full = np.where(sentinel_ids != 0, sentinel_ids, input_ids)
        # IMPORTANT: keep only strictly positive ids to drop pads (0) and fused negatives
        input_ids_compact = input_ids_full[input_ids_full > 0].reshape((batch_size, -1))
        input_ids_compact = np.concatenate(
            [input_ids_compact, np.full((batch_size, 1), self.tokenizer.eos_token_id, dtype=np.int32)],
            axis=-1,
        )
        return input_ids_compact

    def random_spans_noise_mask(self, length):
        orig_length = length
        num_noise_tokens = int(np.round(length * self.noise_density))
        num_noise_tokens = min(max(num_noise_tokens, 1), length - 1)
        num_noise_spans = int(np.round(num_noise_tokens / self.mean_noise_span_length))
        num_noise_spans = max(num_noise_spans, 1)
        num_nonnoise_tokens = length - num_noise_tokens

        def _random_segmentation(num_items, num_segments):
            mask = np.arange(num_items - 1) < (num_segments - 1)
            np.random.shuffle(mask)
            first = np.pad(mask, [[1, 0]])
            seg_id = np.cumsum(first)
            _, seg_len = np.unique(seg_id, return_counts=True)
            return seg_len

        noise_span_lengths = _random_segmentation(num_noise_tokens, num_noise_spans)
        nonnoise_span_lengths = _random_segmentation(num_nonnoise_tokens, num_noise_spans)
        interleaved = np.reshape(np.stack([nonnoise_span_lengths, noise_span_lengths], axis=1), [-1])
        span_starts = np.cumsum(interleaved)[:-1]
        span_start_indicator = np.zeros((length,), dtype=np.int8)
        span_start_indicator[span_starts] = 1
        span_num = np.cumsum(span_start_indicator)
        is_noise = (span_num % 2) == 1
        return is_noise[:orig_length]



# training_args = TrainingArguments()        output_dir="Saved_models/"+args.output_dir,
# per_device_train_batch_size=16, # Adjust based on GPU memory
# gradient_accumulation_steps=2, # To simulate a larger batch size
# max_steps=args.max_step,
# logging_steps=20000,
# save_steps=20000,

# save_total_limit=1,

# dataloader_pin_memory=False,

# bf16=True, # Use mixed precision for faster training and less memory usage

# ddp_find_unused_parameters=False,

# report_to="tensorboard",

# logging_strategy="steps",

# logging_dir="logs/new_log_test",

# load_best_model_at_end=True,

# eval_strategy="steps",

# metric_for_best_model="eval_loss",

# per_device_eval_batch_size=16,

# eval_steps=20000,  

# warmup_steps = args.warm_up_step,

# eval_accumulation_steps=1,

# disable_tqdm=False,

# log_level="info",)



# #optimizer

# optimizer = Adafactor(
#      model.parameters(),
#      lr=args.lr,
#      scale_parameter=False,
#      relative_step=False,
#      warmup_init=False)

# # Create the inverse square root scheduler
# scheduler = get_inverse_sqrt_schedule(
#        optimizer,
#        num_warmup_steps=training_args.warmup_steps,)

# print("trainer init")

# trainer = Trainer(

#     model=model,
#     args=training_args,
#     train_dataset=tokenized_train_dataset,
#     eval_dataset=tokenized_eval_dataset,
#     processing_class=tokenizer,
#     data_collator=data_collator, # You'd put your custom T5 data collator here
#     optimizers=(optimizer, scheduler)
#     )

# print("training started")

# trainer.train()

In [3]:
import os
import gc
import re
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import (
    AutoTokenizer, 
    AutoModelForSeq2SeqLM, 
    # DataCollatorForSeq2Seq, 
    # DataCollatorForLanguageModeling,
    Seq2SeqTrainingArguments, 
    Seq2SeqTrainer
    # AutoModelForMaskedLM,
    # set_seed
    # DataCollatorForT5MLM
)
# from transformers import DataCollatorForLanguageModeling, 
# from transformers import AutoModelForMaskedLM
from sentence_transformers import SentenceTransformer, util
import evaluate

2026-02-03 18:27:52.461187: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770143272.656703      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770143272.714052      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770143273.156600      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770143273.156644      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770143273.156647      55 computation_placer.cc:177] computation placer alr

In [4]:
class Config:
    # Akkadian transliteration contains a lot of noise and many unknown words, so
    # ByT5, which processes text at the character (byte) level rather than the word level, is the strongest choice.
    MODEL_NAME = "google/byt5-small" 
    # MODEL_NAME = "Helsinki-NLP/opus-mt-tc-big-ar-en"
    
    # ByT5 tends to produce longer token sequences, but 512 tokens is enough at the sentence level.
    MAX_LENGTH = 512
    
    BATCH_SIZE = 8       # Adjust depending on GPU memory (on a P100 you can usually go with 8–16).
    EPOCHS = 20
    # EPOCHS = 1
    LEARNING_RATE = 2e-4
    # OUTPUT_DIR = "./arab-engl-akkadian-model"
    OUTPUT_DIR = "./byt5"

In [5]:
# Fix the seed (for reproducibility).
def seed_everything(seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    torch.cuda.manual_seed(seed)
    
seed_everything()

In [6]:
INPUT_DIR = "/kaggle/input/data-exploration"
train_df = pd.read_csv(f"{INPUT_DIR}/gapless_monolingual_train.csv")
test_df = pd.read_csv(f"{INPUT_DIR}/gapless_monolingual_test.csv")

In [7]:
print(f"Original Train Data: {len(train_df)} docs")

Original Train Data: 3453 docs


In [8]:
# Convert to Hugging Face Dataset format & split into Train/Val.
dataset = Dataset.from_pandas(train_df)
# Create a validation set with test_size=0.1.
split_datasets = dataset.train_test_split(test_size=0.1, seed=42)
# After splitting, the keys are 'train' and 'test' (we'll use 'test' as validation).


In [9]:

# ds_mlm = ds.map(tokenize, batched=True)
# ds_mlm = ds_mlm.remove_columns(["labels","text","label_ids"])








In [10]:
# ==========================================
# 3. Tokenization & preprocessing
# ==========================================
tokenizer = AutoTokenizer.from_pretrained(Config.MODEL_NAME)

# Fix the corresponding section in dpc-starter-train.
PREFIX = "Fill in blanks in Akkadian text: "

def preprocess_function(examples):
    inputs = [PREFIX + str(ex) for ex in examples["transliteration"]]
    # targets = [str(ex) for ex in examples["translation"]]
    
    # model_inputs = tokenizer(inputs, max_length=Config.MAX_LENGTH, truncation=True)
    # labels = tokenizer(targets, max_length=Config.MAX_LENGTH, truncation=True)
    
    # model_inputs["labels"] = labels["input_ids"]
    # return model_inputs
    return tokenizer(inputs,truncation=True,max_length=Config.MAX_LENGTH,return_special_tokens_mask=True)

tokenized_train = split_datasets["train"].map(preprocess_function, batched=True)
tokenized_val = split_datasets["test"].map(preprocess_function, batched=True)


tokenizer_config.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/3107 [00:00<?, ? examples/s]

Map:   0%|          | 0/346 [00:00<?, ? examples/s]

In [11]:
# block_size = Config.MAX_LENGTH
# def group_texts(examples): # TODO ACTUALLY USE THIS.
#     # Concatenate all texts.
#     concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
#     total_length = len(concatenated_examples[list(examples.keys())[0]])
#     # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
#     # customize this part to your needs.
#     if total_length >= block_size:
#         total_length = (total_length // block_size) * block_size
#     # Split by chunks of block_size.
#     result = {
#         k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
#         for k, t in concatenated_examples.items()
#     }
#     return result

In [12]:
# ==========================================
# 4. Model training (fine-tuning)
# ==========================================
gc.collect()
torch.cuda.empty_cache()
model = AutoModelForSeq2SeqLM.from_pretrained(Config.MODEL_NAME)
# data_collator = DataCollatorForMaskedLM(tokenizer, model=model)
# data_collator = DataCollatorForLanguageModeling(tokenizer = tokenizer, mlm_probability=0.15, model=model)
# CHATGPT WROTE THIS:
data_collator = DataCollatorForT5MLM(
    # tokenizer=tokenizer,
    # noise_density=0.15,
    # mean_noise_span_length=3.0,

    # IMPORTANT: ByT5 needs shorter lengths
    # input_length=1024,
    # target_length=256,

    # pad_token_id=tokenizer.pad_token_id,
    # decoder_start_token_id=tokenizer.pad_token_id,  # ← FIX
)
# END OF WHAT CHATGPT WROTE


# Metric (chrF++ is part of the competition metric and measures character-level precision/overlap).
metric = evaluate.load("chrf")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple): preds = preds[0]
    
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Ignore -100 in the labels.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"chrf": result["score"]}




args = Seq2SeqTrainingArguments(
    output_dir=Config.OUTPUT_DIR,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=Config.LEARNING_RATE,
    
    # === Key fixes ===
    fp16=False,                     # ★Set to False to prevent a NaN error (required).
    per_device_train_batch_size=4,  # ★fp32 uses more memory, so reduce the batch size (8 -> 4).
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,  # ★To compensate, accumulate gradients to keep the effective batch size at 8.
    # ======================
    
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=Config.EPOCHS,
    predict_with_generate=True,
    logging_steps=10,               # Inspect logs in more detail.
    report_to="none"
)
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=tokenized_train,
    # train_dataset=ds_mlm["unsup"], 
    eval_dataset=tokenized_val,
    # eval_dataset=ds_mlm["train"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

print("Starting Training (FP32 mode)...")
trainer.train()

# See training history
df_log = pd.DataFrame(trainer.state.log_history)

(df_log.dropna(subset=["eval_loss"]).reset_index()["eval_loss"]
    .plot(label="Validation"))
df_log.dropna(subset=["loss"]).reset_index()["loss"].plot(label="Train")

plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(loc="upper right")
plt.show() 


pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/tmp/ipykernel_55/3207647986.py:66: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Starting Training (FP32 mode)...


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (4,) + inhomogeneous part.

In [ ]:
# --- Save Model ---
# Important: the model saved here will be loaded in the next notebook.
trainer.save_model(Config.OUTPUT_DIR)
tokenizer.save_pretrained(Config.OUTPUT_DIR)
print(f"Model saved to {Config.OUTPUT_DIR}")
